In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable

from PIL import Image
import cv2
import albumentations as A

import time
import os
from tqdm.notebook import tqdm

!pip3 install -q segmentation-models-pytorch
!pip3 install -q torchsummary

from torchsummary import summary
import segmentation_models_pytorch as smp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("mps")


In [34]:
#torch.save(sub_images, "/kaggle/input/moblenetv3")
model = torch.load("/kaggle/input/moblenetv3/Unet-Mobilenet_large100.pt",map_location=torch.device("cuda"))

In [35]:
IMAGE_PATH = "/kaggle/input/drone-seg/inputs/inputs/"
MASK_PATH =  "/kaggle/input/drone-seg/semantic_annotations/semantic_annotations/"

In [36]:
n_classes = 17

# Initialize an empty list to store the names of the image files
imageNames = np.array([])

# Iterate through the directories and filenames in the specified IMAGE_PATH
for dirname, _, filenames in os.walk(IMAGE_PATH):
    for filename in filenames:
        # Check if the file has a .png extension
        if filename.endswith('.png'):
            # Remove the file extension and append the filename to the names list
            imageNames = np.append(imageNames, filename.split('.')[0])
print('Total Images: ', len(imageNames))

Total Images:  1785


In [38]:
#split data
X_trainval, X_test = train_test_split(imageNames, test_size=0.1, random_state=19)
X_train, X_val = train_test_split(X_trainval, test_size=0.15, random_state=19)

print('Train Size   : ', len(X_train))
print('Val Size     : ', len(X_val))
print('Test Size    : ', len(X_test))

Train Size   :  1365
Val Size     :  241
Test Size    :  179


In [39]:
class DroneDataset(Dataset):
    
    def __init__(self, img_path, mask_path, X, mean, std, transform=None, patch=False):
        self.img_path = img_path
        self.mask_path = mask_path
        self.X = X
        self.transform = transform
        self.patches = patch
        self.mean = mean
        self.std = std
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = cv2.imread(self.img_path + self.X[idx] + '.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #print()
        #keys.append(self.img_path + self.X[idx] + '.png')
        mask = cv2.imread(self.mask_path + self.X[idx] + '.png', cv2.IMREAD_GRAYSCALE)
        #print(np.unique(mask))
        mask[mask == 255] = 16
        #print(np.asarray(img).shape)
        if self.transform is not None:
            aug = self.transform(image=img, mask=mask)
            img = Image.fromarray(aug['image'])
            #print(np.asarray(img).shape)
            mask = aug['mask']
        
        if self.transform is None:
            img = Image.fromarray(img)
        
        t = T.Compose([T.ToTensor(), T.Normalize(self.mean, self.std)])
        img = t(img)
        mask = torch.from_numpy(mask).long()
            
        return img, mask
    
    def tiles(self, img, mask):

        img_patches = img.unfold(1, 512, 512).unfold(2, 768, 768) 
        img_patches  = img_patches.contiguous().view(3,-1, 512, 768) 
        img_patches = img_patches.permute(1,0,2,3)
        
        mask_patches = mask.unfold(0, 512, 512).unfold(1, 768, 768)
        mask_patches = mask_patches.contiguous().view(-1, 512, 768)
        
        return img_patches, mask_patches
    
   

In [40]:
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

t_train = A.Compose([A.Resize(704, 1056, interpolation=cv2.INTER_NEAREST), A.HorizontalFlip(), A.VerticalFlip(), 
                     A.GridDistortion(p=0.2), A.RandomBrightnessContrast((0,0.5),(0,0.5)),
                     A.GaussNoise()])

t_val = A.Compose([A.Resize(704, 1056, interpolation=cv2.INTER_NEAREST), A.HorizontalFlip(),
                   A.GridDistortion(p=0.2)])

#datasets
train_set = DroneDataset(IMAGE_PATH, MASK_PATH, X_train, mean, std, t_train, patch=False)
val_set = DroneDataset(IMAGE_PATH, MASK_PATH, X_val, mean, std, t_val, patch=False)

#dataloader
batch_size= 3

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)               

In [41]:
#device = torch.device("cuda")
output_master = []
masked_master = []


ground_truth = []
sub_images = []


with torch.no_grad():
    for i, data in enumerate(tqdm(val_loader)):
        #training phase

        image_tiles, mask_tiles = data
        patch = True
        if patch:
            #print(image_tiles.size())
            n_tiles, c, h, w = image_tiles.size()

            image_tiles = image_tiles.view(-1,c, h, w)
            mask_tiles = mask_tiles.view(-1, h, w)

        for i in [0,1,2]:
            try:
                sub_images.append(image_tiles[i][0])
                ground_truth.append(mask_tiles[i])
            except:
                continue




        #plt.show()
        image = image_tiles.to(device); mask = mask_tiles.to(device);
        #forward
        #break
        output = model(image)
        masked = torch.argmax(output, dim=1)

        output_master.append(output)
        for i in [0,1,2]:
            try:
                masked_master.append(masked[i])
            except:
                continue

        output = None
        masked = None
        image_tiles = None
        
        #output = output.cpu().numpy()
        #output_master = np.append(output_master, output)
        #masked = masked.cpu().numpy()
        #masked_master = np.append(masked_master, masked)
        


  0%|          | 0/81 [00:00<?, ?it/s]

In [43]:
torch.save(sub_images, './input_images_list_MobileNetv3.pt')
torch.save(ground_truth, './ground_truth_list_MobileNet_v3.pt')
torch.save(masked_master, './masked_list_MobileNet_v3.pt')